In [439]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from datetime import datetime, timedelta, time
from bokeh import events
from bokeh.io import output_file, output_notebook, curdoc
from bokeh.plotting import figure, show
from bokeh.core.properties import List,Seq
from bokeh.models import CustomJS, ColumnDataSource, LinearAxis, Range1d, Toggle, Slider, DatetimeTickFormatter, Select, Tabs, TabPanel, HoverTool, CrosshairTool, Span, Div, RadioButtonGroup, DatetimeTicker,Text,Title,HelpButton,Tooltip
from bokeh.models.dom import HTML
from bokeh.layouts import row, column, gridplot, layout
from bokeh_realtime_packagev1.functions import *
from bokeh_realtime_packagev1.conditions import *
output_file('testing.html')  # Render to static HTML, or
output_notebook()  # Render inline in a Jupyter Notebook
# from icecream import ic
from iertools.tb import TB
pd.set_option('display.max_rows', 20000)
pd.set_option('display.max_columns',None)
from dateutil.parser import parse

Loading BokehJS ...

## Ocupation panel

## Generando los datos de sonido en dB
Tomamos como referencia los datos del DTHIS del auditorio los modificamos con un random para obtener una serie de datos para cada espacio, descartando el espacio de Atención COFI por cuestiones de confidencialidad.

In [2]:
fecha = parse('2019-01-01 00:05:00')
inicios = [fecha+pd.Timedelta('1D')*i for i in range(365)]
fecha = parse('2019-01-01 23:59:59')
finales = [fecha+pd.Timedelta('1D')*i for i in range(365)]

tmp0= TB(config_file='config_co2.ini',device_name='DTHIS')
sensor = "Nivel Sonoro"

fecha1 = parse("2022-02-02")
#fecha2 = fecha1+pd.Timedelta('1D')
termopares = tmp0.get_df(key=sensor,start_datetime=fecha1)
fecha1 = parse("2022-04-06 19:47:30")
termopares2 = tmp0.get_df(key=sensor,start_datetime=fecha1)
temp = pd.concat([termopares,termopares2])
sonido = [temp['Nivel Sonoro'][i] for i in range(len(temp))]
for j in range(365):
    fechas=[]
    for i in range(287):
        fechas.append(inicios[j]+(pd.Timedelta('5min')*i))
    fechas.append(finales[j])
    joining = pd.DataFrame()
    zones = ['N1AU401', 'N1AU402', 'N1AU403', 'N1AU404', 'N2AU101', 'N2AU102', 'N2AU103', 'N2AU201', 'N2AU202', 'N2AU203', 'PBADM', 'PBCOORCOFI', 'PBCOORDINACION', 'PBSJUNTAS', 'PBSMAESTROS', 'PBVESTIBULO', 'N1LAB', 'N2LAB1', 'N2LAB2', 'PBCAFE']
    for zone in zones:
        tmp=[]
        for i in range(144):
            tmp.append(sonido[i]+np.random.uniform(0,30))
            tmp.append(sonido[i]+np.random.uniform(0,30))
        joining['dB_'+zone]=tmp
    joining['ts']=fechas
    joining['time']=fechas
    joining.set_index('ts',inplace=True)
    if j == 0:
        last = joining.copy()
    else:
        save = pd.concat([last,joining])
        last = save.copy()

["CO2","HR","Humedad Relativa","I","Iluminacion","NPS","Nivel Sonoro","Promedio Pixeles","T","Temperatura","avg","ilum","lpx","max","min","stedv"]


## Generando los datos de CO2
Seguimos un procedimiento similar al anterior, excluyendo de misma manera el espacio de Atención COFI

In [3]:
tmp0= TB(config_file='config_co2.ini',device_name='ESP')
sensor = "s8lp"
fecha2=parse('2023-03-28 06:00:00')
fecha1=fecha2-pd.Timedelta('1D')+pd.Timedelta('10min')
termopares = tmp0.get_df(key=sensor,start_datetime=fecha1,end_datetime=fecha2).resample('10min').mean()
tmp2=[termopares['s8lp'][i] for i in range(143)]
tmp2.append(422.4)
zones = ['N1AU401', 'N1AU402', 'N1AU403', 'N1AU404', 'N2AU101', 'N2AU102', 'N2AU103', 'N2AU201', 'N2AU202', 'N2AU203', 'PBADM', 'PBCOORCOFI', 'PBCOORDINACION', 'PBSJUNTAS', 'PBSMAESTROS', 'PBVESTIBULO', 'N1LAB', 'N2LAB1', 'N2LAB2', 'PBCAFE']
values = []
while len(values) < 105120:
    for value in tmp2:
        values.append(value+np.random.uniform(-2,2))
        values.append(value+np.random.uniform(-2,2))
for zone in zones:
    save['co2_'+zone]=values
    values = [value+np.random.uniform(-1,1) for value in values]

["s8lp","user-agent"]


In [4]:
def do_numbers(n):
    x = np.random.uniform(0,100,n)
    x_int = [int(num) for num in x]
    if sum(x_int) == 0:
        return do_numbers(n)
    else:
        x_int = [float(round(num/sum(x_int)*100)) for num in x_int]
    if sum(x_int) != 100:
        return do_numbers(n)
    else:
        return x_int

In [5]:
for zone in zones:
    tmp1=[]
    tmp2=[]
    tmp3=[]
    for i in range(105120):
        x = do_numbers(3)
        # ic(x)
        tmp1.append(x[0])
        tmp2.append(x[1])
        tmp3.append(x[2])
    save['ilu_sub_'+zone]=tmp1
    save['ilu_u_'+zone]=tmp2
    save['ilu_sob_'+zone]=tmp3
occupation = save.copy()

/tmp/ipykernel_47953/2146373134.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  save['ilu_sob_'+zone]=tmp3


In [6]:
occupation = occupation.truncate(after='2019-01-01 23:59:59')

In [7]:
# occupation.info()
# occupation.to_csv('../data/tb/ambient_quality_data.csv')

In [8]:
dB_zones_au = []
dB_zones_others = []
for zone in occupation.columns:
    if 'dB_' in zone:
        if 'AU' in zone:
            dB_zones_au.append(zone)
        else:
            dB_zones_others.append(zone)
co2_zones=[]
co2_zones_au = []
co2_zones_others = []
for zone in occupation.columns:
    if 'co2_' in zone:
        if 'AU' in zone:
            co2_zones_au.append(zone)
        else:
            co2_zones_others.append(zone)
        co2_zones.append(zone)
sub_zones_au = []
sub_zones_others = []
for zone in occupation.columns:
    if 'ilu_sub_' in zone:
        if 'AU' in zone:
            sub_zones_au.append(zone)
        else:
            sub_zones_others.append(zone)
u_zones_au = []
u_zones_others = []
for zone in occupation.columns:
    if 'ilu_u_' in zone:
        if 'AU' in zone:
            u_zones_au.append(zone)
        else:
            u_zones_others.append(zone)
sob_zones_au = []
sob_zones_others = []
for zone in occupation.columns:
    if 'ilu_sob_' in zone:
        if 'AU' in zone:
            sob_zones_au.append(zone)
        else:
            sob_zones_others.append(zone)

In [9]:
aulas_n = ['N1AU401', 'N1AU402', 'N1AU403', 'N1AU404', 'N2AU101', 'N2AU102', 'N2AU103', 'N2AU201', 'N2AU202', 'N2AU203']
others_n = ['PBADM', 'PBCOORCOFI', 'PBCOORDINACION', 'PBSJUNTAS', 'PBSMAESTROS', 'PBVESTIBULO', 'N1LAB', 'N2LAB1', 'N2LAB2', 'PBCAFE']

co2s_au=[occupation[name][100] for name in co2_zones_au]
dbs_au=[occupation[name][100] for name in dB_zones_au]
subs_au=[occupation[name][100] for name in sub_zones_au]
us_au=[occupation[name][100] for name in u_zones_au]
sobs_au=[occupation[name][100] for name in sob_zones_au]

co2s_others=[occupation[name][100] for name in co2_zones_others]
dbs_others=[occupation[name][100] for name in dB_zones_others]
subs_others=[occupation[name][100] for name in sub_zones_others]
us_others=[occupation[name][100] for name in u_zones_others]
sobs_others=[occupation[name][100] for name in sob_zones_others]

In [10]:
source=ColumnDataSource(occupation)
exceed_dB = np.arange(115,80,-5.)
warning_dB = np.arange(85,55,-5.)
adequate_dB = np.arange(60,-5,-5.)

exceed_co2 = np.arange(480,505,5.)
warning_co2 = np.arange(440,485,5.)
adequate_co2 = np.arange(400,445,5.)
p = figure(y_range=others_n, height=300,
           toolbar_location=None, tools="",width=603,margin=(0,0,0,0))

p.hbar(y=others_n, right=co2s_others,left=0, height=0.6,line_color='black',color='white')

p.xgrid.grid_line_color = None
p.x_range.start = 400
p.x_range.end=500
p.xaxis.axis_label= 'CO2 concentration [ppm]'
p.yaxis.major_label_text_align='right'
p.varea(x=exceed_co2,y1=0,y2=10,color='red',alpha=0.2)
p.varea(x=warning_co2,y1=0,y2=10,color='yellow',alpha=0.2)
p.varea(x=adequate_co2,y1=0,y2=10,color='green',alpha=0.2)

# p.yaxis.major_label_standoff = 100



p2 = figure(y_range=others_n, height=300,width=553,
           toolbar_location=None,y_axis_location='right', tools="",margin=(0,0,0,15))

p2.hbar(y=others_n, right=dbs_others, height=0.6,line_color='black',color='white')

p2.xgrid.grid_line_color = None
p2.x_range.start = 115
p2.x_range.end=0
p2.xaxis.axis_label = 'Sound levels [dB]'
p2.yaxis.major_label_text_color='white'
p2.yaxis.major_label_standoff = 0
p2.yaxis.major_label_text_font_size = '0.5pt'
p2.varea(x=exceed_dB,y1=0,y2=10,color='red',alpha=0.2)
p2.varea(x=warning_dB,y1=0,y2=10,color='yellow',alpha=0.2)
p2.varea(x=adequate_dB,y1=0,y2=10,color='green',alpha=0.2)

p5 = figure(y_range=aulas_n, height=300,width=550,
           toolbar_location=None, tools="",x_axis_type=None,x_range=p.x_range,margin=(0,0,0,55))

p5.hbar(y=aulas_n, right=co2s_au,left=0, height=0.6,line_color='black',color='white')

p5.xgrid.grid_line_color = None
# p5.x_range.start = 0
# p5.x_range.end=500

p5.yaxis.visible = True
p5.varea(x=exceed_co2,y1=0,y2=10,color='red',alpha=0.2)
p5.varea(x=warning_co2,y1=0,y2=10,color='yellow',alpha=0.2)
p5.varea(x=adequate_co2,y1=0,y2=10,color='green',alpha=0.2)

p6 = figure(y_range=aulas_n, height=300,width=550,
           toolbar_location=None, tools="",y_axis_location='right',x_axis_type=None,x_range=p2.x_range,margin=(0,0,0,15))

p6.hbar(y=aulas_n, left=dbs_au, right=0, height=0.6,line_color='black',color='white')

p6.xgrid.grid_line_color = None
p6.x_range.start = 115
p6.x_range.end=0
p6.yaxis.major_label_text_color = 'white'
p6.yaxis.major_label_standoff = 0
p6.yaxis.major_label_text_font_size = '0.5pt'
p6.varea(x=exceed_dB,y1=0,y2=10,color='red',alpha=0.2)
p6.varea(x=warning_dB,y1=0,y2=10,color='yellow',alpha=0.2)
p6.varea(x=adequate_dB,y1=0,y2=10,color='green',alpha=0.2)


p3 = figure(width=1915,height=306,x_axis_type='datetime')
for zona in co2_zones:
    if 'AU' in zona:
        p3.line(y=zona,x='time',source=source,line_color='blue',name=zona)
    elif 'LAB' in zona:
        p3.line(y=zona,x='time',source=source,line_color='red',name=zona)
    else:
        p3.line(y=zona,x='time',source=source,line_color='green',name=zona)
p3.yaxis.axis_label = 'CO2 concentration [ppm]'

aulas=pd.DataFrame()
aulas['subs']=subs_au
aulas['us']=us_au
aulas['sobs']=sobs_au
aulas['zones']=aulas_n
ilus_au=['subs','us','sobs']
p7=figure(y_range=aulas_n, height=300,width=550,toolbar_location=None, tools="",x_axis_type=None,margin=(0,0,0,53))

p7.hbar_stack(ilus_au,y='zones',source=aulas,color=['green','blue','red'],line_color='black')
p7.x_range.start = 0
p7.x_range.end =100

others=pd.DataFrame()
others['subs']=subs_au
others['us']=us_au
others['sobs']=sobs_au
others['zones']=others_n
ilus_others=['subs','us','sobs']
p4=figure(y_range=others_n, height=300,width=603,toolbar_location=None, tools="",x_range=p7.x_range,margin=(0,0,0,0))

p4.hbar_stack(ilus_others,y='zones',source=others,color=['green','blue','red'],line_color='black')
p4.x_range.start = 0
p4.x_range.end = 100
p4.xaxis.axis_label = 'Percentage of surfaces [%]'

perce =[]
for i in range(10):
    perce.append(random.randint(0,100))

df=pd.DataFrame()
df['x']=np.ones(10)*0.5
df['y']=np.array([0.38,1.48,2.58,3.68,4.8,5.9,7,8.1,9.2,10.3])
df['text']=[str(num)+'%' for num in perce]

p8 = figure(height=300,
           toolbar_location=None, tools="",width=40,margin=(0,0,0,0),y_range=Range1d(0,11))

textdf=ColumnDataSource(df)
p8.text(x='x',y='y',text='text',text_align="center",
        text_font_size="10px",
        text_font="opensans",
        text_font_style="normal",source=textdf)
p8.yaxis.visible = False
p8.xaxis.visible = False
p8.grid.visible = False

p9 = figure(height=300,
           toolbar_location=None, tools="",width=40,margin=(0,0,0,0),y_range=Range1d(0,11))

textdf=ColumnDataSource(df)
p9.text(x='x',y='y',text='text',text_align="center",
        text_font_size="10px",
        text_font="opensans",
        text_font_style="normal",source=textdf)
p9.yaxis.visible = False
p9.xaxis.visible = False
p9.grid.visible = False

occupation_panel=layout([[p7,p6,p5,p9],[p4,p2,p,p8],[p3]])

# show(occupation_panel)

## Comfort panel

In [112]:
archivo = "../data/edificio_este/edificio_este.csv"

nombres= np.genfromtxt(archivo,max_rows=1,dtype="U",delimiter=',')

#for i,nombre in enumerate(nombres):
#    print(i,nombre)
ti_zones=[]
rh_zones=[]
all_zones=['N1AU401','N1AU402','N1AU403','N1AU404','N2AU101','N2AU102','N2AU103','N2AU201','N2AU202','N2AU203','N2PASILLO','N2STR','PBADM','PBATENCIONCOFI','PBCOORCOFI','PBCOORDINACION','PBDUCTOCOFI','PBDUCTOSALAS','PBDUCTOSCOM','PBIMP','PBSCOM','PBSJUNTAS','PBSMAESTROS','PBSTR','PBVESTIBULO']
zones1=['N1AU401','N1AU402','N1AU403','N1AU404','N2AU101','N2AU102','N2AU103','N2AU201','N2AU202','N2AU203','PBADM','PBATENCIONCOFI','PBCOORCOFI','PBCOORDINACION','PBSJUNTAS','PBSMAESTROS','PBVESTIBULO']
zMAT=range(2,27,1)
zRAH=range(27,52,1)
nombres[0]='time'
nombres[1]='T_out'
for i in range(len(all_zones)):
    nombres[zMAT[i]]='Ti_'+all_zones[i]
    nombres[zRAH[i]]='RH_'+all_zones[i]
    if all_zones[i] in zones1:
        ti_zones.append(nombres[zMAT[i]])
        rh_zones.append(nombres[zRAH[i]])
        
use_cols=[nombres[0],nombres[1]]+ti_zones+rh_zones
def eplus_fileimport(archivo):
    tmp = pd.read_csv(archivo,names=nombres,skiprows=1,usecols=use_cols)
    tmp.time = tmp.time.str.replace('24:00:00','23:59:59')
    tmp.time = '2019 '+ tmp.time
    tmp.time = pd.to_datetime(tmp.time,format='%Y %m/%d %H:%M:%S')
    tmp.set_index('time',inplace=True)
    return tmp

simulation1 = eplus_fileimport(archivo)

# este = simulation.truncate(after='2019-01-01 23:59:59')

archivo2 = "../data/edificio_oeste/edificio_oeste.csv"

nombres2= np.genfromtxt(archivo,max_rows=1,dtype="U",delimiter=',')


ti_zones=[]
rh_zones=[]
# all_zones=['N1AU401','N1AU402','N1AU403','N1AU404','N2AU101','N2AU102','N2AU103','N2AU201','N2AU202','N2AU203','N2PASILLO','N2STR','PBADM','PBATENCIONCOFI','PBCOORCOFI','PBCOORDINACION','PBDUCTOCOFI','PBDUCTOSALAS','PBDUCTOSCOM','PBIMP','PBSCOM','PBSJUNTAS','PBSMAESTROS','PBSTR','PBVESTIBULO']
zones2=['N1LAB','N2LAB1','N2LAB2','PBCAFE','PBCOCINA']
zMAT=range(2,7,1)
zRAH=range(7,12,1)
nombres2[0]='time'
for i in range(len(zones2)):
    nombres2[zMAT[i]]='Ti_'+zones2[i]
    nombres2[zRAH[i]]='RH_'+zones2[i]
    ti_zones.append(nombres2[zMAT[i]])
    rh_zones.append(nombres2[zRAH[i]])
# for i,nombre in enumerate(nombres2):
#    print(i,nombre)

use_cols=[nombres2[0]]+ti_zones+rh_zones
def eplus_fileimport(archivo):
    tmp = pd.read_csv(archivo,names=nombres2,skiprows=1,usecols=use_cols)
    tmp.time = tmp.time.str.replace('24:00:00','23:59:59')
    tmp.time = '2019 '+ tmp.time
    tmp.time = pd.to_datetime(tmp.time,format='%Y %m/%d %H:%M:%S')
    tmp['datetime']=tmp.time
    tmp.set_index('time',inplace=True)
    tmp['hour']=tmp.index.hour
    return tmp

simulation2 = eplus_fileimport(archivo)

# oeste = simulation.truncate(after='2019-01-01 23:59:59')

com_zones=zones1+zones2

edificio = pd.merge(simulation1,simulation2,left_index=True,right_index=True)
edificio

,T_out,Ti_N1AU401,Ti_N1AU402,Ti_N1AU403,Ti_N1AU404,Ti_N2AU101,Ti_N2AU102,Ti_N2AU103,Ti_N2AU201,Ti_N2AU202,Ti_N2AU203,Ti_PBADM,Ti_PBATENCIONCOFI,Ti_PBCOORCOFI,Ti_PBCOORDINACION,Ti_PBSJUNTAS,Ti_PBSMAESTROS,Ti_PBVESTIBULO,RH_N1AU401,RH_N1AU402,RH_N1AU403,RH_N1AU404,RH_N2AU101,RH_N2AU102,RH_N2AU103,RH_N2AU201,RH_N2AU202,RH_N2AU203,RH_PBADM,RH_PBATENCIONCOFI,RH_PBCOORCOFI,RH_PBCOORDINACION,RH_PBSJUNTAS,RH_PBSMAESTROS,RH_PBVESTIBULO,Ti_N1LAB,Ti_N2LAB1,Ti_N2LAB2,Ti_PBCAFE,Ti_PBCOCINA,RH_N1LAB,RH_N2LAB1,RH_N2LAB2,RH_PBCAFE,RH_PBCOCINA,datetime,hour
time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:05:00,19.000000,20.737269,20.753784,20.728014,20.726742,21.210985,21.263567,21.231804,21.013638,21.056919,21.040199,22.712728,22.278509,22.964886,22.196927,22.162826,22.765537,22.754904,33.255087,33.255870,33.287607,33.293891,32.256980,32.187084,32.227756,32.784541,32.725497,32.740056,27.905886,29.922476,27.269907,30.315929,30.169994,27.552411,28.408749,20.737269,20.753784,20.728014,20.726742,21.210985,21.263567,21.231804,21.013638,21.056919,21.040199,2019-01-01 00:05:00,0
2019-01-01 00:10:00,18.946528,20.690192,20.718537,20.684297,20.684467,21.111872,21.170568,21.134833,20.916034,20.962439,20.943190,22.703548,22.400906,22.903728,22.214769,22.175545,22.738282,22.754523,32.730308,32.739531,32.767215,32.773693,31.857548,31.804583,31.834235,32.515413,32.476076,32.478094,27.924424,29.677474,27.393212,30.238146,30.113593,27.604662,28.437857,20.690192,20.718537,20.684297,20.684467,21.111872,21.170568,21.134833,20.916034,20.962439,20.943190,2019-01-01 00:10:00,0
2019-01-01 00:15:00,18.900000,20.434596,20.481200,20.436725,20.437652,21.018664,21.079505,21.042298,20.906380,20.948225,20.932012,22.680936,22.458244,22.857152,22.157113,22.127758,22.724818,22.746481,32.613620,32.601552,32.639038,32.645349,31.539732,31.495804,31.519579,32.143698,32.128900,32.114236,27.960226,29.534968,27.487752,30.261938,30.138151,27.632850,28.470116,20.434596,20.481200,20.436725,20.437652,21.018664,21.079505,21.042298,20.906380,20.948225,20.932012,2019-01-01 00:15:00,0
2019-01-01 00:20:00,18.853472,20.259779,20.311351,20.265451,20.266371,21.010291,21.069678,21.033395,20.986521,21.022141,21.010112,22.659222,22.417598,22.860375,22.091353,22.073470,22.729378,22.737014,32.935118,32.896078,32.947145,32.951820,31.574921,31.524355,31.552386,32.009314,32.001437,31.981940,27.998346,29.598606,27.489633,30.346998,30.211909,27.628228,28.492951,20.259779,20.311351,20.265451,20.266371,21.010291,21.069678,21.033395,20.986521,21.022141,21.010112,2019-01-01 00:20:00,0
2019-01-01 00:25:00,18.800000,20.039087,20.091833,20.047236,20.047938,20.996118,21.054227,21.018763,21.159542,21.180656,21.177573,22.617691,22.435432,22.809327,21.992764,21.989176,22.728952,22.681675,33.430476,33.360395,33.427219,33.429607,31.650939,31.587817,31.623919,31.721165,31.733301,31.701527,28.074879,29.562183,27.585322,30.475254,30.329766,27.634893,28.580787,20.039087,20.091833,20.047236,20.047938,20.996118,21.054227,21.018763,21.159542,21.180656,21.177573,2019-01-01 00:25:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-31 23:40:00,18.669125,19.515324,19.528423,19.494843,19.497866,20.547833,20.587773,20.558314,20.434131,20.443848,20.437797,22.171654,22.109192,22.634524,21.381665,21.333347,22.430396,22.575200,46.050745,45.996473,46.102937,46.092604,43.334866,43.232809,43.307956,43.929319,43.932306,43.929862,39.672971,39.341367,37.793267,41.695359,41.834227,38.513536,38.120763,19.515324,19.528423,19.494843,19.497866,20.547833,20.587773,20.558314,20.434131,20.443848,20.437797,2019-12-31 23:40:00,23
2019-12-31 23:45:00,17.800000,19.838934,19.846635,19.813358,19.816206,20.571088,20.614711,20.583024,20.266218,20.285777,20.273560,22.207658,22.151986,22.607220,21.533017,21.460284,22.399958,22.562401,43.700396,43.628002,43.750574,43.737832,41.778473,41.645

### Esquema de ventilación
Para llevar a cabo el análisis de ventilación tomaremos en cuenta que las ventanas son de 3x3m y hay 2 por Aula de cada lado, asimismo se considera que hay 1 de cada lado por oficina.

In [113]:
#Cálculo del área de ventanas
for zone in com_zones:
    if 'AU' in zone:
        effective_window_size=18
        tmp1=[]
        tmp2=[]
        for j in range(len(edificio)):
            if edificio.hour[j] > 18:
                tmp1.append(0)
                tmp2.append(0)
            elif edificio.hour[j] > 16:
                tmp1.append((0.5+np.random.uniform(-0.2,0.1))*effective_window_size)
                tmp2.append((0.5+np.random.uniform(-0.2,0.1))*effective_window_size)
            elif edificio.hour[j] > 13:
                tmp1.append((0.75+np.random.uniform(-0.1,0.2))*effective_window_size)
                tmp2.append((0.75+np.random.uniform(-0.1,0.2))*effective_window_size)
            elif edificio.hour[j] > 11:
                tmp1.append((0.5+np.random.uniform(-0.2,0.1))*effective_window_size)
                tmp2.append((0.5+np.random.uniform(-0.2,0.1))*effective_window_size)
            else:
                tmp1.append(0)
                tmp2.append(0)
        edificio['A_inW'+zone]=tmp1
        edificio['A_outW'+zone]=tmp2
        edificio['A_totW'+zone]=edificio[['A_inW'+zone,'A_outW'+zone]].sum(axis=1)
        #ic(tmp1)
        #ic(tmp2)
    else:
        effective_window_size=9
        tmp1=[]
        for j in range(len(edificio)):
            if edificio.hour[j] > 18:
                tmp1.append(0)
            elif edificio.hour[j] > 16:
                tmp1.append((0.5+np.random.uniform(-0.2,0.1))*effective_window_size)
            elif edificio.hour[j] > 13:
                tmp1.append((0.75+np.random.uniform(-0.1,0.2))*effective_window_size)
            elif edificio.hour[j] > 11:
                tmp1.append((0.5+np.random.uniform(-0.2,0.1))*effective_window_size)
            else:
                tmp1.append(0)
        edificio['A_totW'+zone]=tmp1

En este caso para el esquema de ventilación también consideramos que cada uno de los espacios tiene ventilas.
Estas tienen una área efectiva de 1.5x0.2m por espacio.
Primero calculamos las columnas de datos del ángulo de apertura, donde consideramos ángulos de 0° a 90° ó $\pi/2$

Debido a que buscamos que funcionen como sistema de ventilación nocturna, los consideramos prácticamente abiertos de 19:00h a 07:00h y cerrados en cualquier otro caso.

In [114]:
effective_vent_size = 1.5*0.2
for zone in com_zones:
    tmp1=[]
    for j in range(len(edificio)):
        if edificio.hour[j] > 19:
            tmp1.append(effective_vent_size)
        elif edificio.hour[j] > 7:
            tmp1.append(0)
        else:
            tmp1.append(effective_vent_size)
    edificio['A_totV'+zone]=tmp1

Creando temperatura promedio mensual exterior

In [115]:
Mmean=edificio.T_out.resample('M').mean()
jan=np.ones(12*24*31)*Mmean[0]
feb=np.ones(12*24*28)*Mmean[1]
mar=np.ones(12*24*31)*Mmean[2]
apr=np.ones(12*24*30)*Mmean[3]
may=np.ones(12*24*31)*Mmean[4]
jun=np.ones(12*24*30)*Mmean[5]
jul=np.ones(12*24*31)*Mmean[6]
aug=np.ones(12*24*31)*Mmean[7]
sep=np.ones(12*24*30)*Mmean[8]
oct=np.ones(12*24*31)*Mmean[9]
nov=np.ones(12*24*30)*Mmean[10]
dec=np.ones(12*24*31)*Mmean[11]
meanY=np.concatenate((jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec))
edificio['Mmean_Ti']=meanY

In [116]:
edificio['Tn1']=11.9+0.54*edificio.Mmean_Ti
edificio['lsc1']=edificio['Tn1']+1.5
edificio['lic1']=edificio['Tn1']-1.5
for zone in com_zones:
    tmp=edificio['Ti_'+zone][edificio['Ti_'+zone]>edificio.lsc1]-edificio.lsc1
    tmp.fillna(0,inplace=True)
    edificio['GHDC1_'+zone]=tmp*(1./12.)
edificio['Tn2']=13.5+0.54*edificio.Mmean_Ti
edificio['lsc2']=edificio['Tn2']+1.5
edificio['lic2']=edificio['Tn2']-1.5
for zone in com_zones:
    tmp=edificio['Ti_'+zone][edificio['Ti_'+zone]>edificio.lsc2]-edificio.lsc2
    tmp.fillna(0,inplace=True)
    edificio['GHDC2_'+zone]=tmp*(1./12.)
edificio['Tn3']=12.1+0.54*edificio.T_out.mean()
edificio['lsc3']=edificio['Tn3']+1.5
edificio['lic3']=edificio['Tn3']-1.5
for zone in com_zones:
    tmp=edificio['Ti_'+zone][edificio['Ti_'+zone]>edificio.lsc3]-edificio.lsc3
    tmp.fillna(0,inplace=True)
    edificio['GHDC3_'+zone]=tmp*(1./12.)

/tmp/ipykernel_47953/979295302.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  edificio['GHDC3_'+zone]=tmp*(1./12.)
/tmp/ipykernel_47953/979295302.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  edificio['GHDC3_'+zone]=tmp*(1./12.)
/tmp/ipykernel_47953/979295302.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `ne

In [253]:
imodels1=edificio[['Tn1','lsc1','lic1','datetime']].copy()
imodels2=edificio[['Tn2','lsc2','lic2','datetime']].copy()
imodels3=edificio[['Tn3','lsc3','lic3','datetime']].copy()
imodels1.columns = ['Tn','lsc','lic','times']
imodels2.columns = ['Tn','lsc','lic','times']
imodels3.columns = ['Tn','lsc','lic','times']
# lists['ghdc1']=[str(round(edificio['GHDC1_'+zone].iloc[100],1))+' °h' for zone in com_zones]
# lists['ghdc2']=[str(round(edificio['GHDC2_'+zone].iloc[100],1))+' °h' for zone in com_zones]
# lists['ghdc3']=[str(round(edificio['GHDC3_'+zone].iloc[100],1))+' °h' for zone in com_zones]
# titles['ghdc1']=['GH Disconfort cálido']
# titles['ghdc2']=['GH Disconfort cálido']
# titles['ghdc3']=['GH Disconfort cálido']
# ploting=pd.concat([titles,lists])
# imodels1=pd.concat([ploting['ghdc1'],imodels1],ignore_index=True)
# imodels2=pd.concat([ploting['ghdc2'],imodels1],ignore_index=True)
# imodels3=pd.concat([ploting['ghdc3'],imodels1],ignore_index=True)
# imodels1.columns = ['ghdc','Tn','lsc','lic','times']
# imodels2.columns = ['ghdc','Tn','lsc','lic','times']
# imodels3.columns = ['ghdc','Tn','lsc','lic','times']
ds1=ColumnDataSource(imodels1)
ds2=ColumnDataSource(imodels2)
ds3=ColumnDataSource(imodels3)

In [254]:
imodels1

,Tn,lsc,lic,times
time,,,,
2019-01-01 00:05:00,22.482225,23.982225,20.982225,2019-01-01 00:05:00
2019-01-01 00:10:00,22.482225,23.982225,20.982225,2019-01-01 00:10:00
2019-01-01 00:15:00,22.482225,23.982225,20.982225,2019-01-01 00:15:00
2019-01-01 00:20:00,22.482225,23.982225,20.982225,2019-01-01 00:20:00
2019-01-01 00:25:00,22.482225,23.982225,20.982225,2019-01-01 00:25:00
2019-01-01 00:30:00,22.482225,23.982225,20.982225,2019-01-01 00:30:00
2019-01-01 00:35:00,22.482225,23.982225,20.982225,2019-01-01 00:35:00
2019-01-01 00:40:00,22.482225,23.982225,20.982225,2019-01-01 00:40:00
2019-01-01 00:45:00,22.482225,23.982225,20.982225,2019-01-01 00:45:00


In [117]:
edificio = edificio.truncate(after='2019-01-01 23:59:59').copy()

In [118]:
start_time=edificio.index[0].asm8
end_time=edificio.index[-1].asm8

## Consumptiom panel

## Creating water consume data

In [21]:
def water_consume_normal(fechas,param):
    x = np.linspace(norm.ppf(0.01),norm.ppf(0.99),288)
    y = norm.pdf(x)+np.random.uniform(0,0.2,288)
    for i in range(len(fechas)):
        if fechas[i].hour < 6:
            y[i] = 0
        elif fechas[i].hour >= 21:
            y[i] = 0
    consume=y*param
    return consume
def electric_consume_normal(param):
    x = np.linspace(norm.ppf(0.01),norm.ppf(0.99),288)
    y = norm.pdf(x)+np.random.uniform(0,0.2,288)
    consume=y*param
    return consume

In [22]:
fecha = parse('2019-01-01 00:05:00')
inicios = [fecha+pd.Timedelta('1D')*i for i in range(365)]
fecha = parse('2019-01-01 23:59:59')
finales = [fecha+pd.Timedelta('1D')*i for i in range(365)]

for j in range(365):
    fechas=[]
    for i in range(287):
        fechas.append(inicios[j]+(pd.Timedelta('5min')*i))
    fechas.append(finales[j])
    joining = pd.DataFrame()
    water_zones = ['PBBAÑOH','PBBAÑOM','N1BAÑOH','N1BAÑOM','N2BAÑOH','N2BAÑOM','N1LAB','N2LAB1','N2LAB2','PBCAFE','PBBEB1','PBBEB2','N1BEB','N2BEB']
    for zone in water_zones:
        joining['water_'+zone]=water_consume_normal(fechas,500)
    joining['ts']=fechas
    joining['time']=fechas
    joining.set_index('ts',inplace=True)
    if j == 0:
        last = joining.copy()
    else:
        save = pd.concat([last,joining])
        last = save.copy()

In [23]:
fechas=[]
for i in range(287):
    fechas.append(parse('2019-01-01 00:05:00')+(pd.Timedelta('5min')*i))
fechas.append(parse('2019-01-01 23:59:59'))
water_zones = ['PBBAÑOH','PBBAÑOM','N1BAÑOH','N1BAÑOM','N2BAÑOH','N2BAÑOM','N1LAB','N2LAB1','N2LAB2','PBCAFE','PBBEB1','PBBEB2','N1BEB','N2BEB']
data = pd.DataFrame()
for zone in water_zones:
    data['water_'+zone] = water_consume_normal(fechas,500)

tfechas=[]
for i in range(287):
    tfechas.append(parse('2019-01-02 00:05:00')+(pd.Timedelta('5min')*i))
    fechas.append(parse('2019-01-02 00:05:00')+(pd.Timedelta('5min')*i))
fechas.append(parse('2019-01-02 23:59:59'))
tfechas.append(parse('2019-01-02 23:59:59'))
water_zones = ['PBBAÑOH','PBBAÑOM','N1BAÑOH','N1BAÑOM','N2BAÑOH','N2BAÑOM','N1LAB','N2LAB1','N2LAB2','PBCAFE','PBBEB1','PBBEB2','N1BEB','N2BEB']
tmp = pd.DataFrame()
for zone in water_zones:
    tmp['water_'+zone] = water_consume_normal(tfechas,500)
data = pd.concat([data,tmp],ignore_index=True)

tfechas=[]
for i in range(287):
    tfechas.append(parse('2019-01-03 00:05:00')+(pd.Timedelta('5min')*i))
    fechas.append(parse('2019-01-03 00:05:00')+(pd.Timedelta('5min')*i))
tfechas.append(parse('2019-01-03 23:59:59'))
fechas.append(parse('2019-01-03 23:59:59'))
water_zones = ['PBBAÑOH','PBBAÑOM','N1BAÑOH','N1BAÑOM','N2BAÑOH','N2BAÑOM','N1LAB','N2LAB1','N2LAB2','PBCAFE','PBBEB1','PBBEB2','N1BEB','N2BEB']
tmp = pd.DataFrame()
for zone in water_zones:
    tmp['water_'+zone] = water_consume_normal(tfechas,500)
data = pd.concat([data,tmp],ignore_index=True)
data['ts']=fechas
data['time']=fechas
# data.set_index('ts',inplace=True)
# data

## Creating electric dummy consume data

electric_zones = ['N1AU401', 'N1AU402', 'N1AU403', 'N1AU404', 'N2AU101', 'N2AU102', 'N2AU103', 'N2AU201', 'N2AU202', 'N2AU203', 'PBADM', 'PBCOORCOFI', 'PBCOORDINACION', 'PBSJUNTAS', 'PBSMAESTROS', 'PBVESTIBULO', 'N1LAB', 'N2LAB1', 'N2LAB2', 'PBCAFE']
for zone in electric_zones:
    tmp=[]
    i=0
    while i!=365:
        tmp.append(electric_consume_normal(200))
        i=i+1
    tmp_array=np.asarray(tmp)
    reshaped = tmp_array.reshape(105120)
    save['elec_c_'+zone]=reshaped
for zone in electric_zones:
    tmp=[]
    i=0
    while i!=365:
        tmp.append(electric_consume_normal(20))
        i=i+1
    tmp_array=np.asarray(tmp)
    reshaped = tmp_array.reshape(105120)
    save['elec_l_'+zone]=reshaped

In [24]:
# save.to_csv('../data/comsuption_data.csv')

In [25]:
electric_zones = ['N1AU401', 'N1AU402', 'N1AU403', 'N1AU404', 'N2AU101', 'N2AU102', 'N2AU103', 'N2AU201', 'N2AU202', 'N2AU203', 'PBADM', 'PBCOORCOFI', 'PBCOORDINACION', 'PBSJUNTAS', 'PBSMAESTROS', 'PBVESTIBULO', 'N1LAB', 'N2LAB1', 'N2LAB2', 'PBCAFE']
tmp=pd.DataFrame()
for zone in electric_zones:
    tmp['elec_c_'+zone]=electric_consume_normal(200)

tmp2=pd.DataFrame()
for zone in electric_zones:
    tmp2['elec_c_'+zone]=electric_consume_normal(200)
tmp = pd.concat([tmp,tmp2],ignore_index=True)
tmp2=pd.DataFrame()
for zone in electric_zones:
    tmp2['elec_c_'+zone]=electric_consume_normal(200)
tmp = pd.concat([tmp,tmp2],ignore_index=True)
data = pd.concat([data,tmp],axis=1)

tmp=pd.DataFrame()
for zone in electric_zones:
    tmp['elec_l_'+zone]=electric_consume_normal(20)

tmp2=pd.DataFrame()
for zone in electric_zones:
    tmp2['elec_l_'+zone]=electric_consume_normal(20)
tmp = pd.concat([tmp,tmp2],ignore_index=True)
tmp2=pd.DataFrame()
for zone in electric_zones:
    tmp2['elec_l_'+zone]=electric_consume_normal(20)
tmp = pd.concat([tmp,tmp2],ignore_index=True)
data = pd.concat([data,tmp],axis=1)
data.set_index('ts',inplace=True)
# data

## Plotting

In [26]:
baños=[]
labs=[]
cafe=[]
beb=[]
for zone in water_zones:
    if 'BAÑO' in zone:
        baños.append('water_'+zone)
    elif 'LAB' in zone:
        labs.append('water_'+zone)
    elif 'CAFE' in zone:
        cafe.append('water_'+zone)
    elif 'BEB' in zone:
        beb.append('water_'+zone)

data['WBAÑOS']=data[baños].sum(axis=1)
data['WLABS']=data[labs].sum(axis=1)
data['WCAFE']=data[cafe].sum(axis=1)
data['WBEB']=data[beb].sum(axis=1)

In [27]:
cont=[]
labs=[]
ofi=[]
for zone in electric_zones:
    if 'AU' in zone:
        cont.append('elec_c_'+zone)
    elif 'LAB' in zone:
        labs.append('elec_c_'+zone)
    elif 'PB' in zone:
        ofi.append('elec_c_'+zone)

data['CONTACTOSAU']=data[cont].sum(axis=1)
data['EQUIPOLAB']=data[labs].sum(axis=1)
data['EQUIPOOFI']=data[ofi].sum(axis=1)
data['LUCES']=data[['elec_l_'+zone for zone in electric_zones]].sum(axis=1)

In [28]:
wzones=['Baños','Laboratorios','Cafetería','Bebederos']
water_accumulate=pd.DataFrame(index=wzones)
for i in range(3):
    extract=data.iloc[0+288*i:288+288*i]
    baños=[]
    labs=[]
    cafe=[]
    beb=[]
    for zone in water_zones:
        if 'BAÑO' in zone:
            baños.append(extract['water_'+zone].sum())
        elif 'LAB' in zone:
            labs.append(extract['water_'+zone].sum())
        elif 'CAFE' in zone:
            cafe.append(extract['water_'+zone].sum())
        elif 'BEB' in zone:
            beb.append(extract['water_'+zone].sum())
    water_accumulate['dia'+str(i)]=[sum(baños),sum(labs),sum(cafe),sum(beb)]
water_accumulate.rename(columns={'dia0':'Antier','dia1':'Ayer','dia2':'Hoy'},inplace=True)
water_accumulate = water_accumulate.T

In [29]:
ezones=['Contactos Aulas','Equipo Laboratorio','Equipo de Oficina','Luces']
electric_accumulate=pd.DataFrame(index=ezones)
for i in range(3):
    extract=data.iloc[0+288*i:288+288*i]
    cont=[]
    labs=[]
    ofi=[]
    light=[]
    for zone in data.columns:
        if 'elec_c_' in zone:
            if 'AU' in zone:
                cont.append(extract[zone].sum())
            elif 'LAB' in zone:
                labs.append(extract[zone].sum())
            elif 'PB' in zone:
                ofi.append(extract[zone].sum())
        elif 'elec_l_' in zone:
            light.append(extract[zone].sum())
    electric_accumulate['dia'+str(i)]=[sum(cont),sum(labs),sum(ofi),sum(light)]
electric_accumulate.rename(columns={'dia0':'Antier','dia1':'Ayer','dia2':'Hoy'},inplace=True)
electric_accumulate = electric_accumulate.T

In [30]:
dat = ColumnDataSource(data.iloc[0:288])
p = figure(height=530,width=960,toolbar_location=None,tooltips='',x_axis_type='datetime')

p.varea_stack(['WBAÑOS','WLABS','WCAFE','WBEB'],x='ts',source=dat,fill_color=['aqua','darkcyan','slateblue','mediumblue'],legend_label=['Baños','Laboratorios','Cafetería','Bebederos'])

p2 = figure(height=530,width=960,toolbar_location=None,tooltips='',x_axis_type='datetime')

p2.varea_stack(['CONTACTOSAU','EQUIPOLAB','EQUIPOOFI','LUCES'],x='ts',source=dat,fill_color=['darkorange','aquamarine','sienna','gold'],legend_label=['Aulas', 'Laboratorios','Uso general','Iluminación'])

awater= ColumnDataSource(water_accumulate)
p3 = figure(height=530,width=960,toolbar_location=None,tooltips='',x_range=['Antier','Ayer','Hoy'])

p3.vline_stack(wzones,x='index',source=awater,legend_label=wzones,color=['aqua','darkcyan','slateblue','mediumblue'],line_width=5)
p3.circle(x='index',y=ezones[0],size=10)
# p3.circle(x='index',y=ezones[0]+ezones[1],size=10)

aelectric= ColumnDataSource(electric_accumulate)
p4 = figure(height=530,width=960,toolbar_location=None,tooltips='',x_range=['Antier','Ayer','Hoy'])

p4.vline_stack(ezones,x='index',source=aelectric,legend_label=ezones,color=['darkorange','aquamarine','sienna','gold'],line_width=5)

consume_panel=layout([[p,p2],[p3,p4]])
# show(consume_panel)

## Panels

In [462]:
source=ColumnDataSource(occupation)
exceed_dB = np.arange(115,80,-5.)
warning_dB = np.arange(85,55,-5.)
adequate_dB = np.arange(60,-5,-5.)

exceed_co2 = np.arange(480,505,5.)
warning_co2 = np.arange(440,485,5.)
adequate_co2 = np.arange(400,445,5.)
p = figure(y_range=others_n, height=300,
           toolbar_location=None, tools="",width=603,margin=(0,0,0,0))

p.hbar(y=others_n, right=co2s_others,left=0, height=0.6,line_color='black',color='white')

p.xgrid.grid_line_color = None
p.x_range.start = 400
p.x_range.end=500
p.xaxis.axis_label= 'CO2 concentration [ppm]'
p.yaxis.major_label_text_align='right'
p.varea(x=exceed_co2,y1=0,y2=10,color='red',alpha=0.2)
p.varea(x=warning_co2,y1=0,y2=10,color='yellow',alpha=0.2)
p.varea(x=adequate_co2,y1=0,y2=10,color='green',alpha=0.2)
p.grid.visible=False

# p.yaxis.major_label_standoff = 100



p2 = figure(y_range=others_n, height=300,width=553,
           toolbar_location=None,y_axis_location='right', tools="",margin=(0,0,0,15))

p2.hbar(y=others_n, right=dbs_others, height=0.6,line_color='black',color='white')

p2.xgrid.grid_line_color = None
p2.x_range.start = 115
p2.x_range.end=0
p2.xaxis.axis_label = 'Sound levels [dB]'
p2.yaxis.major_label_text_color='white'
p2.yaxis.major_label_standoff = 0
p2.yaxis.major_label_text_font_size = '0.5pt'
p2.varea(x=exceed_dB,y1=0,y2=10,color='red',alpha=0.2)
p2.varea(x=warning_dB,y1=0,y2=10,color='yellow',alpha=0.2)
p2.varea(x=adequate_dB,y1=0,y2=10,color='green',alpha=0.2)
p2.grid.visible=False

p5 = figure(y_range=aulas_n, height=300,width=550,
           toolbar_location=None, tools="",x_axis_type=None,x_range=p.x_range,margin=(0,0,0,55),title='Niveles instantáneos de concentración de CO2',title_location='above')

p5.hbar(y=aulas_n, right=co2s_au,left=0, height=0.6,line_color='black',color='white')

p5.xgrid.grid_line_color = None
# p5.x_range.start = 0
# p5.x_range.end=500

p5.yaxis.visible = True
p5.varea(x=exceed_co2,y1=0,y2=10,color='red',alpha=0.2)
p5.varea(x=warning_co2,y1=0,y2=10,color='yellow',alpha=0.2)
p5.varea(x=adequate_co2,y1=0,y2=10,color='green',alpha=0.2)
p5.grid.visible =False
p5.title.align='left'

p6 = figure(y_range=aulas_n, height=300,width=550,
           toolbar_location=None, tools="",y_axis_location='right',x_axis_type=None,x_range=p2.x_range,margin=(0,0,0,15),title='Niveles instantáneos de presión sonora',title_location='above')

p6.hbar(y=aulas_n, left=dbs_au, right=0, height=0.6,line_color='black',color='white')

p6.xgrid.grid_line_color = None
p6.x_range.start = 115
p6.x_range.end=0
p6.yaxis.major_label_text_color = 'white'
p6.yaxis.major_label_standoff = 0
p6.yaxis.major_label_text_font_size = '0.5pt'
p6.varea(x=exceed_dB,y1=0,y2=10,color='red',alpha=0.2)
p6.varea(x=warning_dB,y1=0,y2=10,color='yellow',alpha=0.2)
p6.varea(x=adequate_dB,y1=0,y2=10,color='green',alpha=0.2)
p6.grid.visible = False
p6.title.align = 'right'


p3 = figure(width=1895,height=306,x_axis_type='datetime',y_range=Range1d(400,520),title='Histórico de concentración de CO2',title_location='below',toolbar_location=None, tooltips='',margin=(10,0,0,0))
for zona in co2_zones:
    if 'AU' in zona:
        p3.line(y=zona,x='time',source=source,line_color='blue',name=zona)
    elif 'LAB' in zona:
        p3.line(y=zona,x='time',source=source,line_color='red',name=zona)
    else:
        p3.line(y=zona,x='time',source=source,line_color='green',name=zona)
p3.yaxis.axis_label = 'CO2 concentration [ppm]'
p3.title.align = 'left'
p3.varea(x='time',y1=0,y2=440,source=source,fill_color='green',fill_alpha=0.2)
p3.varea(x='time',y1=440,y2=480,source=source,fill_color='yellow',fill_alpha=0.2)
p3.varea(x='time',y1=480,y2=520,source=source,fill_color='red',fill_alpha=0.2)
p3.x_range.start = start_time
p3.x_range.end = end_time
p3.xaxis.formatter = DatetimeTickFormatter(hours='%H:%M %P')




aulas=pd.DataFrame()
aulas['subs']=subs_au
aulas['us']=us_au
aulas['sobs']=sobs_au
aulas['zones']=aulas_n
ilus_au=['subs','us','sobs']
p7=figure(y_range=aulas_n, height=300,width=550,toolbar_location=None, tools="",x_axis_type=None,margin=(0,0,0,53),title='Superficies iluminadas obtenido cada 30 min',title_location='above')

p7.hbar_stack(ilus_au,y='zones',source=aulas,color=['green','blue','red'],alpha=0.2,line_color='black')
p7.x_range.start = 0
p7.x_range.end =100
p7.grid.visible = False
p7.title.align = 'left'

others=pd.DataFrame()
others['subs']=subs_au
others['us']=us_au
others['sobs']=sobs_au
others['zones']=others_n
ilus_others=['subs','us','sobs']
p4=figure(y_range=others_n, height=300,width=603,toolbar_location=None, tools="",x_range=p7.x_range,margin=(0,0,0,0))

p4.hbar_stack(ilus_others,y='zones',source=others,color=['green','blue','red'],alpha=0.2,line_color='black')
p4.x_range.start = 0
p4.x_range.end = 100
p4.grid.visible=False


perce =[]
for i in range(10):
    perce.append(random.randint(0,100))

df=pd.DataFrame()
df['x']=np.ones(10)*0.5
df['y']=np.array([0.38,1.48,2.58,3.68,4.8,5.9,7,8.1,9.2,10.3])
df['text']=[str(num)+'%' for num in perce]

p8 = figure(height=300,
           toolbar_location=None, tools="",width=48,margin=(0,0,0,0),y_range=Range1d(0,11))

textdf=ColumnDataSource(df)
p8.text(x='x',y='y',text='text',text_align="center",
        text_font_size="10px",
        text_font="opensans",
        text_font_style="normal",source=textdf)
p8.yaxis.visible = False
p8.xaxis.visible = False
p8.grid.visible = False

p9 = figure(height=300,
           toolbar_location=None, tools="",width=69,margin=(0,0,0,0),y_range=Range1d(0,11),title='Porcentaje de área total de ventanas abiertas',title_location='right')

textdf=ColumnDataSource(df)
p9.text(x='x',y='y',text='text',text_align="center",
        text_font_size="10px",
        text_font="opensans",
        text_font_style="normal",source=textdf)
p9.yaxis.visible = False
p9.xaxis.visible = False
p9.grid.visible = False
p9.title.align = 'right'

help_button1 = HelpButton(tooltip=Tooltip(content=HTML("""
This is a tooltip with additional information.<br />
It can use <b>HTML tags</b> like <a href="https://bokeh.org">links</a>!
"""), position="right"),margin=(0,0,0,120))

help_button2 = HelpButton(tooltip=Tooltip(content=HTML("""
This is a tooltip with additional information.<br />
It can use <b>HTML tags</b> like <a href="https://bokeh.org">links</a>!
"""), position="right"),margin=(0,0,0,470))

help_button3 = HelpButton(tooltip=Tooltip(content=HTML("""
This is a tooltip with additional information.<br />
It can use <b>HTML tags</b> like <a href="https://bokeh.org">links</a>!
"""), position="right"),margin=(0,0,0,630))
occupation_panel=layout([[p7,p6,p5,p9],[p4,p2,p,p8],[help_button1,help_button2,help_button3],[p3]])

# show(occupation_panel)

In [463]:
#Data source 
source=ColumnDataSource(edificio)

#Figura 1 Temperatura al interior
#tooltips=[('Space','$name'),('Temperature','$y °C'),('Date','@x')]
height = Span(dimension="height", line_width=1)
# width = Span(dimension="width", line_width=0.5)
crosshair = CrosshairTool(overlay=height)
p = figure(width=985, height=624,x_axis_type=None,margin=(0,0,0,0),toolbar_location=None)
p.y_range = Range1d(16.,27.)
p.yaxis.axis_label = "Temperature [°C]"
hover1 = HoverTool(
    tooltips=[
        ('Space','$name'),
        #('Temperature','$y °C'),
        ('Hour','@datetime{%H:%M}')
    ]
)
#hover1.tooltips = [('Space','$name'),('Temperature','$y °C'),('Date','@$x')]
hover1.formatters = {'@datetime':'datetime'}
hover1.line_policy = 'nearest'
hover1.mode = 'mouse'
p.add_tools(hover1)
p.add_tools(crosshair)

tn = p.line(y='Tn1',x='time',source=source,line_color='orange',name='Tn',line_width=2,line_dash='dashdot')
lsc=p.line(y='lsc1',x='time',source=source,line_color='red',line_dash='dashed')
lic=p.line(y='lic1',x='time',source=source,line_color='red',line_dash='dashed')
uparea=p.varea(x='time',y1=28,y2='lsc1',source=source,fill_color='gray',fill_alpha=0.2)
downarea=p.varea(x='time',y1='lic1',y2=10,source=source,fill_color='gray',fill_alpha=0.2)
for zona in com_zones:
    if 'AU' in zona:
        p.line(y='Ti_'+zona,x='time',source=source,line_color='blue',name=zona)
    elif 'LAB' in zona:
        p.line(y='Ti_'+zona,x='time',source=source,line_color='red',name=zona)
    else:
        p.line(y='Ti_'+zona,x='time',source=source,line_color='green',name=zona)
p.x_range.start=start_time
p.x_range.end=end_time
#p.legend.click_policy = 'hide'
#p.legend.label_text_font_size = '5pt'
#show(p)

#Figura 2 Humedad Relativa
#tooltips2 =[('Space','$name'),('Relative Humidity','$y %'),('Date','@x')]
p2 = figure(width=985,height=306,x_range=p.x_range,x_axis_type='datetime',toolbar_location=None,margin=(0,0,0,0))
p2.yaxis.axis_label = 'Relative Humidity [%]'
#HoverTool.formatters={}
hover2 = HoverTool(
    tooltips=[
        ('Space','$name'),
        #('Relative Humidity','$y %'),
        ('Hour','@datetime{%H:%M}')
    ]
)
#hover1.tooltips = [('Space','$name'),('Temperature','$y °C'),('Date','@$x')]
hover2.formatters = {'@datetime':'datetime'}
p2.add_tools(hover2)
p2.add_tools(crosshair)
for zona in com_zones:
    if 'AU' in zona:
        p2.line(y='RH_'+zona,x='time',source=source,line_color='blue',name=zona)
    elif 'LAB' in zona:
        p2.line(y='RH_'+zona,x='time',source=source,line_color='red',name=zona)
    else:
        p2.line(y='RH_'+zona,x='time',source=source,line_color='green',name=zona)

p2.xaxis.formatter = DatetimeTickFormatter(hours='%H:%M %P')
p2.harea(x1=parse('2019-01-01 12:05:00'),x2=parse('2019-01-01 12:15:00'),y=np.arange(0,100,1),color='green',alpha=0.2)
p2.y_range.start = 0
p2.y_range.end = 60


lists = pd.DataFrame()
lists['zones']=com_zones
lists['ti']=[str(round(edificio['Ti_'+zone].iloc[100],1)) for zone in com_zones]
lists['rh']=[str(round(edificio['RH_'+zone].iloc[100],0))for zone in com_zones]
lists['aw']=[str(round(edificio['A_totW'+zone].iloc[100]+edificio['A_totV'+zone].iloc[100],1))for zone in com_zones]
lists['ghdc1']=[str(round(edificio['GHDC1_'+zone].iloc[100],1)) for zone in com_zones]
lists['ghdc2']=[str(round(edificio['GHDC2_'+zone].iloc[100],1)) for zone in com_zones]
lists['ghdc3']=[str(round(edificio['GHDC3_'+zone].iloc[100],1)) for zone in com_zones]
lists['x1']=np.ones(22)*0.1
lists['x2']=np.ones(22)*2
lists['x3']=np.ones(22)*3.63
lists['x4']=np.ones(22)*5.4
lists['x5']=np.ones(22)*7.1
lists['y']=np.arange(22,0,-1)

titles=pd.DataFrame()
titles['zones']=['Zona']
titles['ti']=['Temperatura al interior\n[°C]']
titles['rh']=['Humedad Relativa\n[%]']
titles['aw']=['Área de ventanas abierta\n[m³]']
titles['ghdc1']=['GH Disconfort cálido\n[°Ch]']
titles['ghdc2']=['GH Disconfort cálido\n[°Ch]']
titles['ghdc3']=['GH Disconfort cálido\n[°Ch]']
titles['x1']=[0.1]
titles['x2']=[2]
titles['x3']=[3.63]
titles['x4']=[5.4]
titles['x5']=[7.1]
titles['y']=[23]

ploting=pd.concat([titles,lists])

listed = ColumnDataSource(ploting)
p3 = figure(width=900,toolbar_location=None,tooltips='',x_range=Range1d(0,7.9),y_range=Range1d(0,25),margin=(-500,0,0,0),title_location='above')
p3.title = 'Tabla de parámetros'
p3.title.text_font_size = '27px'
p3.title.align = 'center'

p3.text(x='x2',y='y',text='ti',text_align="center",
        text_font_size="14.5px",
        text_font="opensans",
        text_font_style="normal",source=listed)
p3.text(x='x3',y='y',text='rh',text_align="center",
        text_font_size="14.5px",
        text_font="opensans",
        text_font_style="normal",source=listed)
p3.text(x='x4',y='y',text='aw',text_align="center",
        text_font_size="14.5px",
        text_font="opensans",
        text_font_style="normal",source=listed)
ghdc = p3.text(x='x5',y='y',text='ghdc1',text_align="center",
        text_font_size="14.5px",
        text_font="opensans",
        text_font_style="normal",source=listed)
p3.text(x='x1',y='y',text='zones',text_align="left",
        text_font_size="14.5px",
        text_font="opensans",
        text_font_style="normal",source=listed)
p3.yaxis.visible = False
p3.xaxis.visible = False
p3.grid.visible = False

models = ["Humphreys","Humphreys & Nicol","Griffiths"]

radio_group = RadioButtonGroup(labels=models,tags=[ds1,ds2,ds3] ,active=0,width=5,width_policy='fixed',margin=(0,0,0,0))
# # radio_group.js_link('tags',ghdc,'data_source')
# radio_group.js_link('tags',tn,'data_source')
# radio_group.js_link('tags',lsc,'data_source')
# radio_group.js_link('tags',lic,'data_source')
# radio_group.js_link('tags',uparea,'data_source')
# radio_group.js_link('tags',downarea,'data_source')
# radio_group.labels_text_font_size = '16px'


# comfort_panel=layout([[p,radio_group,ghdc],[p2,p3]])

comfort_panel=layout([[p,radio_group],[p2,p3]])

# show(comfort_panel)

In [466]:
dat = ColumnDataSource(data.iloc[0:288])
p = figure(height=450,width=898,toolbar_location=None,tooltips='',x_axis_type='datetime',margin=(0,0,0,62),title='Consumo instantáneo de agua')

p.varea_stack(['WBAÑOS','WLABS','WCAFE','WBEB'],x='ts',source=dat,fill_color=['aqua','darkcyan','slateblue','mediumblue'],legend_label=['Baños','Laboratorios','Cafetería','Bebederos'])
p.xaxis.formatter = DatetimeTickFormatter(hours='%H:%M %P')

p2 = figure(height=450,width=898,toolbar_location=None,tooltips='',x_axis_type='datetime',margin=(0,0,0,22),title='Consumo instantáneo de energía eléctrica')

p2.varea_stack(['CONTACTOSAU','EQUIPOLAB','EQUIPOOFI','LUCES'],x='ts',source=dat,fill_color=['darkorange','aquamarine','sienna','gold'],legend_label=['Aulas', 'Laboratorios','Uso general','Iluminación'])
p2.xaxis.formatter = DatetimeTickFormatter(hours='%H:%M %P')

awater= ColumnDataSource(water_accumulate)
p3 = figure(height=450,width=920,toolbar_location=None,tooltips='',x_range=['Antier','Ayer','Hoy'],margin=(0,0,0,40),title='Consumo acumulado actual de agua y de días previos',title_location='below')

p3.vbar_stack(wzones,x='index',source=awater,legend_label=wzones,color=['aqua','darkcyan','slateblue','mediumblue'],width=0.5)
p3.circle(x='index',y=ezones[0],source=awater,fill_color='black',size=10)
p3.legend.orientation = 'horizontal'
p3.y_range.start = 0
p3.y_range.end = 5.6e+5
p3.legend.location = 'top_center'
# p3.circle(x='index',y=ezones[0]+ezones[1],size=10,source=awater)

aelectric= ColumnDataSource(electric_accumulate)
p4 = figure(height=450,width=920,toolbar_location=None,tooltips='',x_range=['Antier','Ayer','Hoy'],title='Consumo acumulado eléctrico actual y de días previos',title_location='below')

p4.vbar_stack(ezones,x='index',source=aelectric,legend_label=ezones,color=['darkorange','aquamarine','sienna','gold'],width=0.5)
p4.legend.orientation = 'horizontal'
p4.y_range.start = 0
p4.y_range.end = 4.6e+5
p4.legend.location = 'top_center'

consume_panel=layout([[p2,p],[p4,p3]])

In [467]:
tab1 = TabPanel(child=comfort_panel,title="Comfort panel")
tab2 = TabPanel(child=occupation_panel,title="Environment Quality panel")
tab3= TabPanel(child=consume_panel,title='Consumption panel')
tabs = Tabs(tabs=[ tab1, tab2, tab3 ])
show(tabs)